In [6]:
import wandb
from tqdm import tqdm
entity = "garrettebaker"
project_name = "procgen"
run_id = "554qi9z6"

api = wandb.Api()

run_path = f"{entity}/{project_name}/{run_id}"
run = api.run(run_path)

files = run.files()

# download files in https://wandb.ai/garrettebaker/procgen/runs/554qi9z6/files/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19
download_dir = "logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19"
for file in tqdm(files):
    if file.name.startswith(download_dir):
        file.download(replace=True, root=download_dir)

100%|██████████| 2006/2006 [34:45<00:00,  1.04s/it] 


In [3]:
import os
import glob
import torch
from procgen_tools import maze, visualization, models
import matplotlib.pyplot as plt
import imageio
from tqdm import tqdm
model_files = glob.glob("/workspace/maze-values-dev-interp/procgen-tools-dev-interp/experiments/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/*.pth")
print(model_files)
# sort by modification time
model_files.sort(reverse=False)
frames_dir = "/workspace/frames"
i = 0
os.makedirs(frames_dir, exist_ok=True)
for model_file in tqdm(model_files):
    # loading to torch
    # Load the model file and print its keys
    checkpoint = torch.load(model_file, map_location=torch.device('cpu'))
    if "state_dict" in checkpoint:
        checkpoint['model_state_dict'] = checkpoint.pop("state_dict")
    torch.save(checkpoint, model_file)
    # try to load to cuda, if that doesn't work try to load to cpu
    try:
        policy = models.load_policy(model_file, 15, torch.device('cpu'))  # Adjust parameters as needed
    except:
        print("Loading to cuda failed, loading to cpu...")
        policy = models.load_policy(model_file, 15, torch.device('cpu'))

    venv = maze.create_venv(1, start_level=0, num_levels=1)
    vf_original = visualization.vector_field(venv, policy)
    
    # Save each plot as an image
    frame_filename = os.path.join(frames_dir, f"frame_{i}.png")
    i += 1
    visualization.plot_vf(vf_original)
    plt.savefig(frame_filename)
    plt.close()


['/workspace/maze-values-dev-interp/procgen-tools-dev-interp/experiments/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/model_100007936.pth', '/workspace/maze-values-dev-interp/procgen-tools-dev-interp/experiments/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/model_10010624.pth', '/workspace/maze-values-dev-interp/procgen-tools-dev-interp/experiments/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/model_100204544.pth', '/workspace/maze-values-dev-interp/procgen-tools-dev-interp/experiments/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/model_100401152.pth', '/workspace/maze-values-dev-interp/procgen-tools-dev-interp/experiments/logs/procgen/maze_aisc/hard-run/seed_712_13-01-2024_07-29-19/log

100%|██████████| 1000/1000 [36:38<00:00,  2.20s/it]


In [2]:
import os
import glob
import re
from moviepy.editor import ImageSequenceClip

frames_dir = "/workspace/frames"

# Function to extract the frame number from the filename
def extract_number(f):
    s = re.findall("\d+", f)
    return (int(s[0]) if s else -1, f)

# Sort frame files numerically
frame_files = sorted(glob.glob(os.path.join(frames_dir, "*.png")), key=extract_number, reverse=True)

# Create video clip
clip = ImageSequenceClip(frame_files, fps=4)  # Adjust fps
clip.write_videofile('/workspace/policy_evolution.mp4', codec='libx264')

# Display the video (in Jupyter notebooks)
from IPython.display import Video
Video('/workspace/policy_evolution.mp4')


Moviepy - Building video /workspace/policy_evolution.mp4.
Moviepy - Writing video /workspace/policy_evolution.mp4



Moviepy - Done !
Moviepy - video ready /workspace/policy_evolution.mp4


In [10]:
from IPython.display import Video

Video("/workspace/policy_evolution.mp4")

In [ ]:
# Create GIF
frame_files = sorted(glob.glob(os.path.join(frames_dir, "*.png")), key=lambda x: os.path.getmtime(x))
images = [imageio.imread(filename) for filename in frame_files]
imageio.mimsave('policy_evolution.gif', images, fps=4)  # Adjust fps

from IPython.display import Image
Image(filename='policy_evolution.gif')

In [ ]:
import torch as t
from procgen_tools import maze, vfield, models

# Load pretrained policy
policy = models.load_policy('trained_models/maze_I/model_rand_region_5.pth', 15, t.device('cpu'))
venv = maze.create_venv(1, start_level=0, num_levels=1)
vf_original = vfield.vector_field(venv, policy)
vfield.plot_vf(vf_original)
plt.show()